In [11]:
# Import libraries for evaluation
import pandas as pd
import numpy as np
import random
from surprise import SVD, Dataset, Reader
from surprise import accuracy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [12]:
# Load dataset and split into train/test
df = pd.read_csv('../data/processed/stress_relief_data.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print('Training set size:', len(train_df))
print('Test set size:', len(test_df))

Training set size: 80
Test set size: 20


In [13]:
# Random Baseline
def random_baseline():
    activities = ['meditation', 'music', 'journaling', 'breathing', 'walking']
    return random.choice(activities)

# Popularity Baseline
def popularity_baseline(df):
    activity_scores = df.groupby('activity')['feedback'].sum().sort_values(ascending=False)
    return activity_scores.index[0]

# SVD Model
def train_svd_model(train_df):
    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(train_df[['user_id', 'activity', 'feedback']], reader)
    trainset = data.build_full_trainset()
    model = SVD(n_factors=10, random_state=42)
    model.fit(trainset)
    return model

def recommend_svd(user_id, model, activities, n=3):
    predictions = [(activity, model.predict(user_id, activity).est) for activity in activities]
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [activity for activity, _ in predictions[:n]]

# TF-IDF Content-Based Model
def train_tfidf_model(train_df):
    df_liked = train_df[train_df['feedback'] == 1].copy()
    df_liked['profile'] = df_liked['mood'] + ' ' + df_liked['activity']
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_liked['profile'])
    return tfidf, tfidf_matrix, df_liked

def content_based_recommend(mood, tfidf, tfidf_matrix, df_liked, n=3):
    mood_vector = tfidf.transform([mood])
    similarities = cosine_similarity(mood_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-n:][::-1]
    return df_liked.iloc[top_indices]['activity'].tolist()

In [14]:
# LinUCB Bandit Implementation
class LinUCBBandit:
    def __init__(self, arms, context_dim, alpha=1.0):
        self.arms = arms
        self.context_dim = context_dim
        self.alpha = alpha
        self.A = {arm: np.identity(context_dim) for arm in arms}
        self.b = {arm: np.zeros(context_dim) for arm in arms}

    def select_arm(self, context):
        ucb_values = {}
        for arm in self.arms:
            A_inv = np.linalg.inv(self.A[arm])
            theta = np.dot(A_inv, self.b[arm])
            ucb = np.dot(theta, context) + self.alpha * np.sqrt(np.dot(context.T, np.dot(A_inv, context)))
            ucb_values[arm] = ucb
        return max(ucb_values, key=ucb_values.get)

    def update(self, arm, context, reward):
        self.A[arm] += np.outer(context, context)
        self.b[arm] += reward * context

def get_context(mood, stress_level):
    mood_vec = [1 if mood == 'anxious' else 0, 1 if mood == 'calm' else 0]
    stress_vec = [1 if stress_level == 'high' else 0, 1 if stress_level == 'low' else 0]
    return np.array(mood_vec + stress_vec)

In [15]:
# Evaluate Precision@3 for static models
def precision_at_k(test_df, recommendations, k=3):
    hits = 0
    total = 0
    for _, row in test_df.iterrows():
        user_id = row['user_id']
        mood = row['mood']
        true_activity = row['activity']
        true_feedback = row['feedback']
        if true_feedback == 1:  # Only evaluate where true feedback is 1
            total += 1
            if true_activity in recommendations(user_id, mood):
                hits += 1
    return hits / total if total > 0 else 0

# Train models
activities = ['meditation', 'music', 'journaling', 'breathing', 'walking']
svd_model = train_svd_model(train_df)
tfidf, tfidf_matrix, df_liked = train_tfidf_model(train_df)

# Evaluate Random Baseline
random_precision = precision_at_k(test_df, lambda user_id, mood: [random_baseline() for _ in range(3)])
print(f'Random Baseline Precision@3: {random_precision:.3f}')

# Evaluate Popularity Baseline
popularity_activity = popularity_baseline(train_df)
popularity_precision = precision_at_k(test_df, lambda user_id, mood: [popularity_activity for _ in range(3)])
print(f'Popularity Baseline Precision@3: {popularity_precision:.3f}')

# Evaluate SVD
svd_precision = precision_at_k(test_df, lambda user_id, mood: recommend_svd(user_id, svd_model, activities))
print(f'SVD Precision@3: {svd_precision:.3f}')

# Evaluate TF-IDF
tfidf_precision = precision_at_k(test_df, lambda user_id, mood: content_based_recommend(mood, tfidf, tfidf_matrix, df_liked))
print(f'TF-IDF Precision@3: {tfidf_precision:.3f}')

Random Baseline Precision@3: 0.667
Popularity Baseline Precision@3: 0.167
SVD Precision@3: 0.583
TF-IDF Precision@3: 0.417


In [16]:
# Evaluate LinUCB Bandit (Cumulative Reward)
def evaluate_bandit(test_df):
    bandit = LinUCBBandit(activities, context_dim=4, alpha=1.0)
    cumulative_reward = 0
    for _, row in test_df.iterrows():
        context = get_context(row['mood'], row['stress_level'])
        arm = bandit.select_arm(context)
        reward = 1 if (arm == row['activity'] and row['feedback'] == 1) else 0
        bandit.update(arm, context, reward)
        cumulative_reward += reward
    return cumulative_reward / len(test_df) if len(test_df) > 0 else 0.0

bandit_reward = evaluate_bandit(test_df)
print(f'LinUCB Bandit Average Reward: {bandit_reward:.3f}')

LinUCB Bandit Average Reward: 0.100
